<a href="https://colab.research.google.com/github/sinhajiya/NLP/blob/main/Viterbi_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/noisy_test_data.txt

--2025-03-18 16:35:15--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375849 (367K) [text/plain]
Saving to: ‘train_data.txt.11’

train_data.txt.11   100%[===================>] 367.04K  --.-KB/s    in 0.03s   

2025-03-18 16:35:15 (11.8 MB/s) - ‘train_data.txt.11’ saved [375849/375849]

--2025-03-18 16:35:15--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP 

In [ ]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            sentence = []
            for token in line.strip().split():
                word, tag = token.rsplit('/', 1)  # Split word and tag
                sentence.append((word, tag))
            data.append(sentence)
    return data

# Load train and test data from files
train_data_file = '/content/train_data.txt'  # Path to your training data file
test_data_file = '/content/test_data.txt'    # Path to your test data file
noisy_test_data_file = '/content/noisy_test_data.txt'  # Path to your noisy test data file

train_data = load_data(train_data_file)
test_data = load_data(test_data_file)
noisy_test_data = load_data(noisy_test_data_file)

# Print a sample from the training data
print(train_data[0])

[('He', 'PRON'), ('let', 'VERB'), ('her', 'PRON'), ('tell', 'VERB'), ('him', 'PRON'), ('all', 'PRT'), ('about', 'ADP'), ('the', 'DET'), ('church', 'NOUN'), ('.', '.')]


In [ ]:
len(train_data)

2000

###Evaluation
An example of evaluation:

In [ ]:
sentence = [('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]
predicted_tags = ['DET', 'NOUN', 'CONJ', 'NOUN', 'ADP', 'PRON', 'VERB', 'NOUN', '.', 'CONJ', 'NOUN', 'ADJ', '.']
true_tags = ('VERB', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'CONJ', 'NOUN', 'ADJ', '.')
correct = 0
total = 0
correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
total += len(true_tags)
accuracy = correct / total

print(f"Baseline Viterbi Accuracy: {accuracy * 100:.2f}%")
# print(f"Viterbi with Noise Handling Accuracy: {accuracy * 100:.2f}%")  # similarly calculate for Noise Handling


Baseline Viterbi Accuracy: 76.92%


In [ ]:
import numpy as np
import pandas as pd

## HMM SETUP

In [ ]:
def extract_data(data):
  tags = (set() ) # POS tags
  vocab = (set())   # Words in corpos

  for i in data:
    for word, tag in i:
        tags.add(tag)
        vocab.add(word)
  return tags, vocab

In [ ]:
# ELEMENTS OF AN HMM MODEL
tags, vocab = extract_data(train_data)
tags_index = {tags: i for i, tags in enumerate(tags)}
no_tags = len(tags)
vocab_index = {vocab: i for i, vocab in enumerate(vocab)}
vocab_size = len(vocab)

In [ ]:
tags

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [ ]:
def initialize (alpha, no_tags=no_tags, vocab_size=vocab_size):
  start = np.full((no_tags), alpha)  # Initial state probabilities
  tag_transition = np.full((no_tags, no_tags),alpha) # Tag transition: P(t_i|P_i-1)
  emissions =  np.full((no_tags, vocab_size),alpha)  # Word likelihood probability (emission)  P(w_i|t_i)
  return start, tag_transition, emissions

In [ ]:
def form_count_tables(data,alpha):
  start, tag_transition, emissions = initialize(alpha)
  for i in data:
    prev_tag = None
    for word, tag in i:
        tag = tags_index[tag]
        word_index = vocab_index[word]

        emissions[tag][word_index] += 1

        if prev_tag is not None:
            tag_transition[prev_tag][tag] += 1
        else:
            start[tag] += 1  # First word in sentence
        prev_tag = tag
  return start, tag_transition, emissions

In [ ]:
def printtable(table, T=True):
    if T:
        index, columns = tags_index, tags_index
    else:
        index, columns = tags_index, vocab_index

    return(pd.DataFrame(table, index, columns))

In [ ]:
def prob(table, axis=1):
  total_counts = np.sum(table, axis=axis, keepdims=True)
  total_counts[total_counts == 0] = 1
  return table/total_counts

In [ ]:
def form_prob_table (start, tag_transition, emissions):
  start_probs = prob(start, 0)
  transition_prob = prob(tag_transition)
  emission_prob = prob(emissions)
  return start_probs, transition_prob, emission_prob

## Viterbi Implementation

In [ ]:
def viterbi(sentence, setoftags, start_probs , transition_prob , emission_prob):

# Initializing values needed
    tags = list(setoftags)
    no_tags = len(tags)
    no_words = len(sentence)
    unknown_prob = 1e-4  # Small probability for unknown words
    delta = np.zeros((no_tags, no_words))
    shi = np.zeros((no_tags, no_words), dtype=int)


# Delta and shi for the first word
    word = sentence[0] # First word in the sentence

    for i, tag in enumerate(tags):
        tag_index = tags_index[tag]
        if word in vocab_index:
            delta[tag_index, 0] = start_probs[tag_index] * emission_prob[tag_index][vocab_index[word]]
        else:
            delta[tag_index, 0] = start_probs[tag_index] * unknown_prob
        shi[tag_index, 0] = 0  # No previous state in the first column

# Delta and Shi for the rest of the sentence:
    for i in range(1, no_words):  # For a word in the sentence
        word = sentence[i]

        for j, current_tag in enumerate(tags):  # For a single tag:
            max_prob = -float('inf')
            best_prev_tag = 0
            current_tag_index = tags_index[current_tag]

            for k, prev_tag in enumerate(tags):  # For the tag of the word before this word
                prev_tag_index = tags_index[prev_tag]

                if word in vocab_index:
                    prob = (delta[k, i-1] * transition_prob[prev_tag_index][current_tag_index] * emission_prob[current_tag_index][vocab_index[word]])
                else:
                    prob = (delta[k, i-1] * transition_prob[prev_tag_index][current_tag_index] * unknown_prob)


                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = prev_tag_index

            delta[j, i] = max_prob  # store the probability of tag j for word i
            shi[j, i] = best_prev_tag
            # print(f"Word: {word}, Tag: {tags[j]}, Max Prob: {max_prob}")


# Trace back:
    best_last_tag = np.argmax(delta[:, no_words-1])
    best_path = [best_last_tag]

    for i in range(no_words-1, 0, -1):
        best_last_tag = (shi[best_last_tag, i])
        best_path.insert(0, best_last_tag)

    best_tag_sequence = [tags[i] for i in best_path]

    return best_tag_sequence

## TESTING

In [ ]:
def algo_accuracy(test_data, start_probs,transition_prob, emission_prob, tags = tags, B=True):
  correct = 0
  total = 0

  for sentence in test_data:
      words = [word for word, _ in sentence]  # Extract words
      true_tags = [tag for _, tag in sentence]  # Extract true tags


      predicted_tags = viterbi(words, tags, start_probs, transition_prob, emission_prob)

      correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
      total += len(true_tags)

  accuracy = correct / total
  if B:
    print(f"Viterbi Algorithm Accuracy: {accuracy:.2%}")
  else:
    print(f"Viterbi with Noise Handling Accuracy: {accuracy * 100:.2f}%")

In [ ]:
def test(alpha, test_data = test_data, noisy_test_data = noisy_test_data, train_data=train_data):

  start, tag_transition, emissions = form_count_tables(train_data, alpha)
  start_probs, transition_prob, emission_prob = form_prob_table(start, tag_transition, emissions)

  algo_accuracy(test_data, start_probs,transition_prob, emission_prob)
  algo_accuracy( noisy_test_data, start_probs,transition_prob, emission_prob)


In [ ]:
print("WITHOUT SMOOTHING")
test(alpha = 0)

WITHOUT SMOOTHING
Viterbi Algorithm Accuracy: 89.64%
Viterbi Algorithm Accuracy: 81.31%


In [ ]:
print("With k=0.5 smoothing")
test(0.5)

With k=0.5 smoothing
Viterbi Algorithm Accuracy: 85.82%
Viterbi Algorithm Accuracy: 77.85%


In [ ]:
print("With k=1 Smoothing")
test(alpha = 1)

With k=1 Smoothing
Viterbi Algorithm Accuracy: 83.45%
Viterbi with Noise Handling Accuracy: 75.74%


In [ ]:
print("With k=0.3 Smoothing")
test(alpha = 0.3)

With k=0.3 Smoothing
Viterbi Algorithm Accuracy: 87.31%
Viterbi with Noise Handling Accuracy: 79.18%
